In [1]:
import os
os.environ['TFHUB_CACHE_DIR']='/workspace/sentence-embedding/use-model'
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
config = tf.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
tf.compat.v1.Session(config=config) 
from tqdm import tqdm
import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow_text

Device mapping: no known devices.


In [2]:
def paragraph_splited(doc,doc_id):
    para_id = 0
    splited = []
    data_frame = []
    for idx,d in enumerate(doc):
        for d_p in d.split('\n'):
            if d_p != '':
                data_frame.append([doc_id[idx],d_p])
                splited.append(d_p)
        para_id+=1
    data_frame = pd.DataFrame(data_frame ,columns =['para_id', 'para'])
    return splited,data_frame

In [31]:
mode = 'train'
q_lan = ['ar','de','es','hi','vi','zh','ru','bn','fi','ja','ko','te','fr','th','el','ro','tr']

corpus = 'MLQA'

path = f'data_preprocess/{corpus}/{mode}/'
top_start = 1  
top_end = 3

df_question = {}
df_teacher = {}
lan_now = []
df_paragraph = {}
df_doc = {}
for lan in q_lan:
    try:
        question_temp = pd.read_csv(f'data_preprocess/{corpus}/{mode}/{corpus.lower()}_question_en-{lan}.csv')
        question_temp_t = pd.read_csv(f'data_preprocess/{corpus}/{mode}/{corpus.lower()}_question_en-{lan}_en.csv')
        doc_temp = pd.read_csv(f'data_preprocess/{corpus}/{mode}/{corpus.lower()}_doc_en-{lan}.csv')

        df_question.update({
            f'en-{lan}': question_temp
        })
        df_teacher.update({
            f'en-{lan}': question_temp_t
        })
        df_doc.update({
            f'en-{lan}': doc_temp
        })
        
        
        df_paragraph.update({
            f'en-{lan}':pd.read_csv(f'data_preprocess/{corpus}/{mode}/{corpus.lower()}_para_en-{lan}.csv')
        })
        
        lan_now.append(lan)
    except:
        pass
    
q_lan = lan_now


# para-level

In [32]:
context_mode = 'para'
for idx,lan in enumerate(q_lan[:]): #.drop_duplicates(subset=['question'])
    print(f'Lan:{idx+1}/{len(q_lan)}')
    df_merged = df_question[f'en-{lan}'].copy()
    df_merged['question_y'] = df_teacher[f'en-{lan}']['question'].to_list()
    df_merged = df_merged.drop_duplicates(subset=['question_y'])
    df_merged = df_merged.drop_duplicates(subset=['question'])
    df_merged = df_merged.rename(columns={'question': 'student','question_y': 'teacher'})
    
    df_merged = df_merged.merge(df_paragraph[f'en-{lan}'],left_on='paragraph_id',right_on='para_id')
    df_merged = df_merged[['student','teacher','para']]
    df_merged.columns = ['student','teacher','doc']
    df_merged.dropna(inplace=True)
    df_merged.drop_duplicates(inplace=True)
    df_merged.to_csv(f'{path}distillation/distillation_para_en-{lan}.csv',index=False)


Lan:1/6
Lan:2/6
Lan:3/6
Lan:4/6
Lan:5/6
Lan:6/6


# doc-level

In [16]:
context_mode = 'para'
for idx,lan in enumerate(q_lan[:]): #.drop_duplicates(subset=['question'])
    print(f'Lan:{idx+1}/{len(q_lan)}')
    df_merged = df_question[f'en-{lan}'].copy()
    df_merged['question_y'] = df_teacher[f'en-{lan}']['question'].to_list()
    df_merged = df_merged.drop_duplicates(subset=['question_y'])
    df_merged = df_merged.drop_duplicates(subset=['question'])
    df_merged = df_merged.rename(columns={'question': 'student','question_y': 'teacher'})
    df_merged = df_merged.merge(df_doc,left_on='doc_id',right_on='doc_id')
    df_merged = df_merged[['student','teacher','doc']]
    df_merged.dropna(inplace=True)
    df_merged.drop_duplicates(inplace=True)
    df_merged.to_csv(f'{path}distillation/distillation_en-{lan}.csv',index=False)


Lan:1/7
Lan:2/7
Lan:3/7
Lan:4/7
Lan:5/7
Lan:6/7
Lan:7/7
